# MoBioFP - Fingerphoto Recognition with Fingerip Semantic Segmentation

## Import Python libraries

In [ ]:
import cv2
import mobiofp
import fingerprint_enhancer
import matplotlib.pyplot as plt

from pathlib import Path

## Define global constants

In [ ]:
DATA_DIR = "../data"
PROCESSED_DIR = DATA_DIR + '/processed'
IMAGE_PATH = DATA_DIR + '/raw/samples/1_i_1_n_1.jpg'
MODELS_DIR = "../models"
MODEL_CHECKPOINT_PATH = MODELS_DIR + "MODEL_PLACEHOLDER"

## Load a U-Net pre-trained model

In [ ]:
model = mobiofp.Segment()
model.load(MODEL_CHECKPOINT_PATH)
model.info()

## Read RGB sample image

In [ ]:
image = cv2.imread(IMAGE_PATH)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = mobiofp.rotate_image(image, 90)

mobiofp.plot_image(image, title=f"Original Image: {image.shape}")

## Fingertip segmentation using U-Net pre-trained model

In [ ]:
mask = model.predict(image)

plt.figure(figsize=(15, 15))
plt.subplot(1, 3, 1)
plt.imshow(image)
plt.title(f"Original Image: {image.shape}")
plt.axis("off")
plt.subplot(1, 3, 2)
plt.imshow(mask, cmap="gray")
plt.title("Predicted Fingertip Mask")
plt.axis("off")
plt.subplot(1, 3, 3)
plt.imshow(image)
plt.imshow(mask, cmap="jet", alpha=0.5)
plt.title("Mask Overlay")
plt.axis("off")
plt.tight_layout()
plt.show()

## Fingertip ROI Extraction

In [ ]:
bbox = mobiofp.extract_roi(mask)
mobiofp.plot_img_bbox(image, bbox, bbox_title="Fingertip")

fingertip = mobiofp.crop_image(image, bbox)
fingertip_mask = mobiofp.crop_image(mask, bbox)

## Fingertip Enhancement (Grayscale conversion, Bilateral Filter, CLAHE)

In [ ]:
fingertip = cv2.cvtColor(fingertip, cv2.COLOR_RGB2GRAY)

mobiofp.plot_img_hist(fingertip, title="Fingertip Histogram")

In [ ]:
# Normalize image
fingertip = cv2.normalize(fingertip, None, 0, 255, cv2.NORM_MINMAX)

# Apply bilateral filter
fingertip = cv2.bilateralFilter(fingertip, 7, 50, 50)

# Apply CLAHE
fingertip = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8)).apply(fingertip)

mobiofp.plot_img_hist(fingertip, title="Bilateral Filter + CLAHE")

## Fingertip Binarization

In [ ]:
# Remove background
fingertip = cv2.bitwise_and(fingertip, fingertip, mask=fingertip_mask)

# Apply Mean Adaptive Threshold
binary = cv2.adaptiveThreshold(fingertip, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 2)

mobiofp.plot_images([fingertip, binary], ["Fingertip", "Adaptive Threshold"], 1, 2)

## Convert fingerphoto (fingertip) to fingerprint

In [ ]:
fingerprint = mobiofp.to_fingerprint(binary)

mobiofp.plot_image(fingerprint, title="Fingerprint")

## Fingerprint Enhancement

In [ ]:
fingerprint = fingerprint_enhancer.enhance_Fingerprint(fingerprint)

mobiofp.plot_image(fingerprint, title="Enhanced Fingerprint")

## Save Fingerprint

In [ ]:
output_file_path = PROCESSED_DIR + "/" + Path(IMAGE_PATH).stem + ".png"

print(f"Saving fingerprint to {output_file_path}")
cv2.imwrite(output_file_path, fingerprint)